## Imports

In [3]:
import tensorflow as tf
from tensorflow import keras

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.src.legacy.preprocessing.image import ImageDataGenerator

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

from tqdm.notebook import tqdm

%matplotlib inline

## Main part

In [25]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.002, momentum=0.8), loss='binary_crossentropy', metrics=['accuracy'])
    return model


model = create_model()
print(model)
model.summary()

<Sequential name=sequential_3, built=True>


/home/eduard/mambaforge/envs/mix/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
data_gen = ImageDataGenerator(rescale=1./255)

train_generator = data_gen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = data_gen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [27]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.5688 - loss: 0.6918 - val_accuracy: 0.6269 - val_loss: 0.6368
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 119ms/step - accuracy: 0.6788 - loss: 0.5933 - val_accuracy: 0.5771 - val_loss: 0.6961
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.7034 - loss: 0.5588 - val_accuracy: 0.6617 - val_loss: 0.6098
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.7119 - loss: 0.5421 - val_accuracy: 0.5821 - val_loss: 0.7431
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.6936 - loss: 0.5385 - val_accuracy: 0.6816 - val_loss: 0.6123
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.7374 - loss: 0.4932 - val_accuracy: 0.6468 - val_loss: 0.6360
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - accuracy: 0.7285 - loss: 0.5234 - val_accuracy: 0.6617 - val_loss: 0.5921
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.7903 - loss: 0.4722 - val_accuracy: 0.

In [30]:
training_accuracies = history.history['accuracy']
training_losses = history.history['loss']
validation_accuracies = history.history['val_accuracy']
validation_losses = history.history['val_loss']

np.median(training_accuracies), np.std(training_losses)

(0.7237499952316284, 0.06828226506067676)

In [34]:
augment_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_train_generator = augment_data_gen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

augmented_history = model.fit(
    augmented_train_generator,
    epochs=10,
    validation_data=test_generator
)

augmented_training_accuracies = augmented_history.history['accuracy']
augmented_training_losses = augmented_history.history['loss']
augmented_validation_accuracies = augmented_history.history['val_accuracy']
augmented_validation_losses = augmented_history.history['val_loss']

Found 800 images belonging to 2 classes.
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 185ms/step - accuracy: 0.6344 - loss: 0.7247 - val_accuracy: 0.6965 - val_loss: 0.5814
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 189ms/step - accuracy: 0.6359 - loss: 0.6195 - val_accuracy: 0.7015 - val_loss: 0.5619
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 187ms/step - accuracy: 0.6795 - loss: 0.6047 - val_accuracy: 0.6965 - val_loss: 0.5884
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 187ms/step - accuracy: 0.6617 - loss: 0.6098 - val_accuracy: 0.7264 - val_loss: 0.5530
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 191ms/step - accuracy: 0.7009 - loss: 0.5520 - val_accuracy: 0.6965 - val_loss: 0.5848
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 186ms/step - accuracy: 0.7089 - loss: 0.5657 - val_accuracy: 0.7264 - val_loss: 0.5550
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 187ms/step - accuracy: 0.7053 - loss: 0.5863 - val_accuracy: 0.7463 - val_loss: 0.5413
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 189ms/step - accuracy:

In [43]:
# test_generator.reset()
# predictions = model.predict(test_generator)

np.mean(augmented_validation_losses), np.mean(augmented_validation_accuracies[-5:])

(0.5676334500312805, 0.7134328365325928)